In [1]:
import sys; sys.path.append("..")
from importlib import reload
import persist_to_disk as ptd
import os
ptd.config.set_project_path(os.path.abspath("../"))
import pandas as pd

from _settings import GEN_PATHS

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def mask_format2(s,flag):
    if pd.isnull(flag): return s
    flag = int(flag)
    assert flag in {0,3}
    if flag & 1 > 0:
        s = "\\textbf{%s}"%s
    return s

In [3]:
import pipeline.summ as summ
import utils.presentation as upre

num_gens = 5

curve, setting = 'auroc', 'mlga~neg_mlgc'

temp = 0.5
all_data = {}

x1 = 'attnnll'
x2 = 'attnnll_nexttoken'
datasets = ['triviaqa_new', 'coqa_new', 'nq_open_new']
for data in datasets:
    for model in ['llama2-13b','gemma-7b',  'mistral-7b']:
        for transfer_dataset in [None] + ['coqa_new']:
            if transfer_dataset == data: continue
            tdf = {}
            for seed in range(10):
                tdf[seed] = summ.cached_summ(summ._clean_path(GEN_PATHS[temp][data][model]), clean=True,
                                           seed=seed, setting=setting, curve=curve, num_gens=num_gens,
                                        acc_name='moe', transfer_dataset=transfer_dataset)
            if len(tdf):
                all_data[(data, model, transfer_dataset)] = pd.DataFrame(tdf).T.rename(columns={x1:f"{x1}@1", x2:f"{x2}@1"})
next(iter(all_data.values()))

,self_prob,degree|agreement_w,acc,blind,attnnll_nexttoken|all,attnnll|all,nll|norm,sar,nll|unnorm,attnnll_nexttoken@1,...,attnnll@9,attnnll@3,attnnll@6,attnnll@8,attnnll@4,attnnll@5,semanticEntropy|norm,semanticEntropyFROMattnnll@10,oracle,_cnt
0,0.648721,0.816194,0.824008,0.824008,0.861496,0.876897,0.877766,0.878343,0.881039,0.887152,...,0.896239,0.896258,0.896307,0.896386,0.896594,0.896946,0.897564,0.904095,1.0,8489.0
1,0.648282,0.821093,0.826486,0.826486,0.861406,0.876778,0.877779,0.877910,0.881513,0.887333,...,0.894084,0.893970,0.894725,0.894348,0.893558,0.894463,0.898677,0.903733,1.0,8495.0
2,0.649023,0.818299,0.827505,0.827505,0.859812,0.875521,0.876905,0.877274,0.880388,0.890863,...,0.896229,0.896754,0.897395,0.896473,0.896990,0.897572,0.897443,0.903908,1.0,8493.0
3,0.651003,0.819315,0.825191,0.825191,0.864055,0.879880,0.881024,0.881432,0.883427,0.894555,...,0.900448,0.902702,0.901745,0.900859,0.903006,0.902418,0.900073,0.906727,1.0,8495.0
4,0.647957,0.820149,0.827054,0.827054,0.861727,0.877617,0.878675,0.879494,0.883839,0.893354,...,0.894660,0.895327,0.894455,0.894785,0.894518,0.894205,0.900465,0.906250,1.0,8494.0
5,0.646400,0.816467,0.826998,0.826998,0.860831,0.876607,0.877950,0.878133,0.881576,0.892671,...,0.897260,0.898832,0.898438,0.897501,0.898124,0.898418,0.898634,0.905218,1.0,8497.0
6,0.647628,0.815712,0.823758,0.823758,0.861124,0.877004,0.878003,0.878674,0.880407,0.892691,...,0.895482,0.898078,0.896411,0.895770,0.897257,0.896465,0.897476,0.903386,1.0,8494.0
7,0.647414,0.817243,0.824929,0.824929,0.862995,0.878709,0.880102,0.880176,0.882945,0.893249,...,0.899689,0.897530,0.897984,0.899324,0.897637,0.897126,0.900058,0.906662,1.0,8488.0
8,0.650392,0.812161,0.827440,0.827440,0.862361,0.878370,0.879062,0.880062,0.882577,0.893429,...,0.899248,0.898677,0.899552,0.899440,0.900052,0.899849,0.898692,0.904906,1.0,8484.0
9,0.644779,0.817360,0.826148,0.826148,0.861933,0.877207,0.878272,0.879338,0.881413,0.889995,...,0.896903,0.892658,0.895839,0.895929,0.895436,0.895781,0.899280,0.905507,1.0,8490.0


## Main

In [4]:
def _filter_methods(x):
    def _good(s):
        if s in {'oracle', 'blind'}:
            return curve == 'auarc'
        if s == 'acc': return False
        return s in {'attnnll@10', 'attnnll_nexttoken@10',
               'nll|norm', 'nll|unnorm', 'sar',
               'degree|agreement_w', 'self_prob'}
    return x.reindex(columns=[_ for _ in x.columns if _good(_)])
tdfs = {(data, model): _ for (data, model, transfer_dataset), _ in all_data.items() if transfer_dataset is None}
summs = {_[0]: upre.summarize_mean_std_pval(_filter_methods(_[1]), paired=True) for _ in tdfs.items()}
next(iter(summs.values()))

,count,mean,std,rank,pval
attnnll@10,10,0.897045,0.001885,7.0,1.000000e+00
attnnll_nexttoken@10,10,0.896097,0.001783,6.0,1.966336e-02
nll|unnorm,10,0.881913,0.001220,5.0,1.142784e-10
sar,10,0.879084,0.001255,4.0,2.714228e-12
nll|norm,10,0.878554,0.001223,3.0,1.091156e-12
degree|agreement_w,10,0.817399,0.002546,2.0,5.798080e-14
self_prob,10,0.648160,0.001812,1.0,3.295824e-20


In [5]:
reload(upre)
_df, _mask = upre.create_printable_df({f"{data}({model})": _ for (data, model), _ in summs.items()},
                                          create_ser_fn=upre.create_printable_ser_2,
                                          scale=100, pval=0.01) #0.49)
upre.printable_df_to_latex(_df.T, _mask.T, pad=False, skip_header=False, mask_format = mask_format2)

  & Deg(E) & P(true) & SL & SL(norm) & TokenSAR & CSL & CSL-Next\\
\midrule
trivia(llama2) & 81.74$\pm$0.25 & 64.82$\pm$0.18 & 88.19$\pm$0.12 & 87.86$\pm$0.12 & 87.91$\pm$0.13 & \textbf{89.70$\pm$0.19} & \textbf{89.61$\pm$0.18}\\
trivia(gemma) & 84.00$\pm$0.19 & 81.82$\pm$0.18 & 88.72$\pm$0.11 & 88.11$\pm$0.08 & 88.09$\pm$0.08 & \textbf{89.71$\pm$0.14} & 89.42$\pm$0.11\\
trivia(mistral) & 81.85$\pm$0.30 & 68.78$\pm$0.28 & 88.81$\pm$0.15 & 88.64$\pm$0.14 & 88.74$\pm$0.13 & \textbf{90.76$\pm$0.18} & \textbf{90.73$\pm$0.15}\\
coqa(llama2) & 69.93$\pm$0.57 & 53.64$\pm$0.41 & 69.50$\pm$0.40 & 72.59$\pm$0.44 & 72.78$\pm$0.47 & \textbf{73.34$\pm$0.74} & \textbf{73.36$\pm$0.57}\\
coqa(gemma) & 70.03$\pm$0.68 & 55.99$\pm$0.42 & 70.83$\pm$0.46 & 71.96$\pm$0.57 & 72.38$\pm$0.55 & \textbf{73.30$\pm$0.57} & \textbf{73.64$\pm$0.63}\\
coqa(mistral) & 69.84$\pm$0.52 & 52.33$\pm$0.42 & 68.97$\pm$0.38 & 70.60$\pm$0.38 & 70.87$\pm$0.41 & \textbf{71.79$\pm$0.75} & \textbf{71.91$\pm$0.63}\\
nq(llama2) & 71

## Uncertainty

In [6]:
def _filter_methods(x):
    def _good(s):
        if s in {'oracle', 'blind'}:
            return curve == 'auarc'
        if s == 'acc': return False
        #if 'degree' in s: return True
        if 'semanticEntropy' not in s: return False
        return True
    return x.reindex(columns=[_ for _ in x.columns if _good(_)])#[x.index.map(_good)]
tdfs = {(data, model): _ for (data, model, transfer_dataset), _ in all_data.items() if transfer_dataset is None}
summs = {_[0]: upre.summarize_mean_std_pval(_filter_methods(_[1]), paired=True) for _ in tdfs.items()}
next(iter(summs.values()))

,count,mean,std,rank,pval
semanticEntropyFROMattnnll@10,10,0.905039,0.001239,3.0,1.000000e+00
semanticEntropy|norm,10,0.898836,0.001124,2.0,1.158109e-11
semanticEntropy|unnorm,10,0.893263,0.001153,1.0,1.015540e-13


In [7]:
reload(upre)
_df, _mask = upre.create_printable_df({f"{data}({model})": _ for (data, model), _ in summs.items()},
                                          create_ser_fn=upre.create_printable_ser_2,
                                          scale=100, pval=0.05) #0.49)
upre.printable_df_to_latex(_df.T, _mask.T, pad=False, skip_header=False, mask_format = mask_format2)

  & SE(norm) & SE & SE+CSL\\
\midrule
trivia(llama2) & 89.88$\pm$0.11 & 89.33$\pm$0.12 & \textbf{90.50$\pm$0.12}\\
trivia(gemma) & 90.33$\pm$0.10 & 90.02$\pm$0.12 & \textbf{90.75$\pm$0.13}\\
trivia(mistral) & 90.35$\pm$0.12 & 89.78$\pm$0.15 & \textbf{91.13$\pm$0.13}\\
coqa(llama2) & \textbf{75.26$\pm$0.33} & 72.50$\pm$0.34 & \textbf{75.58$\pm$0.51}\\
coqa(gemma) & \textbf{74.81$\pm$0.48} & 72.95$\pm$0.48 & \textbf{75.08$\pm$0.50}\\
coqa(mistral) & 74.06$\pm$0.41 & 71.54$\pm$0.32 & \textbf{74.56$\pm$0.51}\\
nq(llama2) & 74.13$\pm$0.51 & 69.62$\pm$0.38 & \textbf{76.33$\pm$0.50}\\
nq(gemma) & 77.93$\pm$0.58 & 73.67$\pm$0.77 & \textbf{79.50$\pm$0.52}\\
nq(mistral) & 75.71$\pm$0.40 & 71.85$\pm$0.49 & \textbf{78.66$\pm$0.35}\\


## Trasnfer

In [8]:

def _filter_methods(x):
    def _good(s):
        if 'attnnll' in s:
            return s.endswith("@10") and 'tropy' not in s
        if s == 'nll|norm': return True
        return False
    return x.reindex(columns=[_ for _ in x.columns if _good(_)])#[x.index.map(_good)]

tdfs = {(data, model): _ for (data, model, transfer_dataset), _ in all_data.items() if transfer_dataset == 'coqa_new'}
summs = {_[0]: upre.summarize_mean_std_pval(_filter_methods(_[1]), paired=True) for _ in tdfs.items()}
next(iter(summs.values()))

reload(upre)
_df, _mask = upre.create_printable_df({f"{data}({model})": _ for (data, model), _ in summs.items()},
                                          create_ser_fn=upre.create_printable_ser_2,
                                          scale=100, pval=0.05) #0.49)
upre.printable_df_to_latex(_df.T, _mask.T, pad=False, skip_header=False, mask_format = mask_format2)


  & SL(norm) & CSL & CSL-Next\\
\midrule
trivia(llama2) & 87.86$\pm$0.12 & \textbf{88.59$\pm$0.85} & 88.37$\pm$0.87\\
trivia(gemma) & 88.11$\pm$0.08 & \textbf{89.03$\pm$0.61} & 87.43$\pm$0.94\\
trivia(mistral) & 88.64$\pm$0.14 & \textbf{89.70$\pm$1.43} & 88.46$\pm$1.93\\
nq(llama2) & 69.48$\pm$0.50 & \textbf{70.94$\pm$1.66} & 70.10$\pm$2.09\\
nq(gemma) & 75.81$\pm$0.65 & \textbf{77.21$\pm$1.06} & 74.99$\pm$1.30\\
nq(mistral) & \textbf{71.06$\pm$0.54} & \textbf{72.71$\pm$3.32} & \textbf{72.27$\pm$2.72}\\
